In [1]:
import sqlite3

import pandas as pd

In [46]:
import re

In [8]:
topic_df = pd.read_csv("data/topic_df.csv")

In [4]:
conn = sqlite3.connect('sql/database.sqlite')

In [5]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('Emails',), ('Persons',), ('Aliases',), ('EmailReceivers',)]


In [13]:
query = "SELECT * FROM Emails;"
pd.read_sql_query(query, conn).head(2)

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,,"Sullivan, Jacob J <Sullivan11@state.gov>",,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,,,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,,,,,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...


In [14]:
query = "SELECT * FROM Persons;"
pd.read_sql_query(query, conn).head(2)

,Id,Name
0,1,111th Congress
1,2,AGNA USEMB Kabul Afghanistan


In [82]:
query = "SELECT * FROM Aliases;"
alias = pd.read_sql_query(query, conn)
alias.head(2)

,Id,Alias,PersonId
0,1,111th congress,1
1,2,agna usemb kabul afghanistan,2


In [16]:
query = "SELECT * FROM EmailReceivers;"
pd.read_sql_query(query, conn).head(2)

,Id,EmailId,PersonId
0,1,1,80
1,2,2,80


In [40]:
query = "SELECT Emails.Id, EmailReceivers.PersonId, Persons.Name, Emails.MetadataDateSent, Emails.ExtractedBodyText \
FROM Emails, Persons, EmailReceivers WHERE Emails.Id = EmailReceivers.EmailId AND EmailReceivers.PersonId = Persons.Id \
AND Emails.SenderPersonId=80 AND Persons.Name NOT LIKE '%@%' AND Emails.Id IN(SELECT MIN(Id) FROM Emails GROUP BY ExtractedBodyText);"
hilary_sent = pd.read_sql_query(query, conn)

In [41]:
hilary_sent.shape

(1936, 5)

In [43]:
hilary_sent.head()

,Id,PersonId,Name,MetadataDateSent,ExtractedBodyText
0,5,81,Huma Abedin,2011-03-11T05:00:00+00:00,"H <hrod17@clintonemail.com>\nFriday, March 11,..."
1,6,185,Robert Russo,2012-09-12T04:00:00+00:00,Pis print.\n-•-...-^\nH < hrod17@clintonernail...
2,8,81,Huma Abedin,2011-03-11T05:00:00+00:00,"H <hrod17@clintonemail.corn>\nFriday, March 11..."
3,21,87,Jake Sullivan,2012-09-12T04:00:00+00:00,"H <hrod17@clintonernaii.com›\nWednesday, Septe..."
4,22,185,Robert Russo,2012-09-12T04:00:00+00:00,Pis print.\nH < hrod17@clintoriernail.corn>\nW...


In [77]:
def remove_n(t):
    t = t.replace('\n'," ")
    t = re.sub(r"[\w]+@[\.\w]+", "", t)
    t = re.sub(r"[0-2]?[0-9]:[0-6][0-9] PM", "", t)
    t = t.replace('H <>'," ")
    t = t.replace('H < >'," ")
    return t

In [78]:
hilary_sent["ExtractedBodyText"] = hilary_sent["ExtractedBodyText"].apply(lambda x: remove_n(x))

In [83]:
hilary_sent.head()

,Id,PersonId,Name,MetadataDateSent,ExtractedBodyText
0,5,81,Huma Abedin,2011-03-11T05:00:00+00:00,"Friday, March 11, 2011 Huma Abedin Fw: H: L..."
1,6,185,Robert Russo,2012-09-12T04:00:00+00:00,"Pis print. -•-...-^ Wednesday, September 12,..."
2,8,81,Huma Abedin,2011-03-11T05:00:00+00:00,"Friday, March 11, 2011 Huma Abedin Fw: H: L..."
3,21,87,Jake Sullivan,2012-09-12T04:00:00+00:00,"H <› Wednesday, September 12, 2012 ' Fw: Fwd:..."
4,22,185,Robert Russo,2012-09-12T04:00:00+00:00,"Pis print. Wednesday, September 12, 2012 °°..."


In [90]:
hilary_sent[hilary_sent["ExtractedBodyText"].duplicated()].groupby(["Name","ExtractedBodyText"]). first()

Id  \
Name                       ExtractedBodyText                                          
Andrew Shapiro             Importance: High Pls read Brian's message and a...   407   
Anne-Marie Slaughter       AMS and Jack----AMS's comments and edits about ...  1334   
                           Are we still planning for me to deliver a speec...  2356   
Cherie Blair               Thanks for your concern I'm on the way to Qatar...  7074   
Cheryl Mills               As I have said repeatedly, I do not want to see...  1551   
                           Can we try to change the Sri Lanka call and res...   392   
                           Good work.                                          7425   
                           Huma Abedin Also food and water?                    6635   
                           I can't believe the govt is closed again. I gue...  6918   
                           I have a time sensitive matter to discuss re Gu...   507   
                           I just got back and will be up for awhile.          7187   
                           Lona---Could we be ready to do on or before 12/...  1163   
                           Ok to do conf call 1300 Sunday and call w Lady ...  6628   
                           Thx. Pls keep me updated.                            374   
                           W/whom? I thought she wanted to speak w me Sund...  6629   
                           When can we talk again? I'll be tied up from 5-...  1467   
                           You did a fabulous job and everybody knew this ...  7561   
Daniel Baer                 B6 Thanks for all your hard work on this diffi...  1218   
David Axelrod              Thx so much for your help--and for Ernie Banks!...   612   
Ellen Tauscher             Lavrov thanked me for sending Ellen to Geneva a...  7226   
Harold Hongju Koh          Good catch, Jeff! Yes--two issues. There will b...  1425   
                           I want to debrief you on my call w Davutogla an...  6817   
                           Jeff/Harold-- I spoke w Kouchener who will be m...  1414   
Huma Abedin                Am I talking w him at 7:30? 1                       7660   
                           Anne-Marie sent me a memo yesterday about her c...   758   
                           As I now understand it, I'm supposed to talk to...   787   
                           Could we do Tuesday after the 8:45?                  816   
                           Could we do the Women FMs and Heads of State as...   933   
                           Could we move to 8:30?                              7826   
                           Did we regret this?                                 2308   
...                                                                             ...   
cheryl.mills jake.sullivan Do you know anything about this--from any angle?     334   
glantz.gina                I would love to see you on Dec 14th as you pass...  1041   
huma abed in               Clinton Is?-over all it reflects your amazing s...   959   
huma abedin b6             The following is from an observer who I think i...  1037   
hume abedin                When can I call him?                                6578   
jake.sullivar              Can you send me the language we prefer in a res...   435   
                           Do you have the two memos that he has prepared ...   331   
                           I would also like an updated list of all the FM...   306   
jake.sullivar preines      Here's what she said: Sounds like a tuff day in...  7554   
jake.sulliyan              Could you work w Derek, Philippe and others to ...   311   
jilotylc©state.gov         Pls print 3 copies.                                 7699   
markjpenr                  Mark--thanks for the offer of help. I'd appreci...  7192   
mtorrey1                   Marty-- Thanks for the update and pls let me kn...  1040   
oscar flores               Ns print.                                           